#### Using a plain descriptor

In cases where an SQL query more elaborate than what `column_property()` or `hybrid_property` can provide must be emitted, a __regular Python function accessed as an attribute__ can be used, assuming the expression __only needs to be available__ on an _already-loaded instance_. The function is decorated with Python's own `@property` decorator __to mark it as a read-only attribute__. Within the function, `object_session()` is used to _locate_ the `Session` corresponding to the current object, which is then used to emit a query.

In [3]:
from sqlalchemy import Column, ForeignKey, Integer, String, select, func
from sqlalchemy.orm import declarative_base, object_session

In [2]:
Base = declarative_base()

In [ ]:
class Address(Base):
    __tablename__ = "address"
    
    id = Column(Integer, primary_key=True)
    user_id = Column(Integer, ForeignKey("user.id"))

In [4]:
class User(Base):
    __tablename__ = "user"
    
    id = Column(Integer, primary_key=True)
    firstname = Column(String(50))
    lastname = Column(String(50))
    
    @property
    def address_count(self):
        return object_session(self).scalar(
            select(func.count(Address.id)).
            where(Address.user_id == self.id)
        )

The _plain descriptor approach_ is useful as a __last resort__, but is __less performant__ in the usual case than _both the hybrid and column property approaches_, in that it needs to emit a SQL query upon each access.